<h1><center>Le classifieur SVM</center></h1>


### Importer les bibliothéques necessaires 

In [22]:
import numpy as np
import pandas as pd

#### 1. Importer votre DataSet

In [23]:
df = pd.read_csv('accent-mfcc-data-1.csv')

##### Print

In [24]:
df.head()


,language,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
0,ES,7.071476,-6.512900,7.650800,11.150783,-7.657312,12.484021,-11.709772,3.426596,1.462715,-2.812753,0.866538,-5.244274
1,ES,10.982967,-5.157445,3.952060,11.529381,-7.638047,12.136098,-12.036247,3.491943,0.595441,-4.508811,2.332147,-6.221857
2,ES,7.827108,-5.477472,7.816257,9.187592,-7.172511,11.715299,-13.847214,4.574075,-1.687559,-7.204041,-0.011847,-6.463144
3,ES,6.744083,-5.688920,6.546789,9.000183,-6.924963,11.710766,-12.374388,6.169879,-0.544747,-6.019237,1.358559,-6.356441
4,ES,5.836843,-5.326557,7.472265,8.847440,-6.773244,12.677218,-12.315061,4.416344,0.193500,-3.644812,2.151239,-6.816310


#### 2. Checker les variables quantitatives/qualitatives et les valeurs manquantes 

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   language  329 non-null    object 
 1   X1        329 non-null    float64
 2   X2        329 non-null    float64
 3   X3        329 non-null    float64
 4   X4        329 non-null    float64
 5   X5        329 non-null    float64
 6   X6        329 non-null    float64
 7   X7        329 non-null    float64
 8   X8        329 non-null    float64
 9   X9        329 non-null    float64
 10  X10       329 non-null    float64
 11  X11       329 non-null    float64
 12  X12       329 non-null    float64
dtypes: float64(12), object(1)
memory usage: 33.5+ KB


In [26]:
# Quantitatives
df.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
count,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000,329.000000
mean,5.645227,-4.270814,2.635319,7.200207,-5.648733,9.810236,-9.408053,5.117328,-1.229432,-2.362288,2.430833,-3.980415
std,5.105302,3.514350,3.635323,4.310095,4.596180,3.624654,2.484117,2.650608,3.634849,5.042034,3.478467,2.985879
min,-6.067831,-14.972962,-6.186583,-8.844231,-15.656596,-3.528964,-15.365665,-2.873862,-15.510974,-11.429178,-13.664104,-13.724103
25%,2.065603,-6.515764,0.137042,4.700874,-8.417684,8.643563,-11.120860,3.482167,-1.749082,-5.083522,1.197789,-5.941409
50%,4.954206,-4.252512,2.029268,7.804680,-6.786670,10.379330,-9.710399,4.843103,-0.389970,-3.323147,3.169703,-4.374334
75%,9.793648,-1.560250,4.178026,10.129165,-4.463308,11.784360,-7.989370,6.588931,0.779993,-1.506037,4.673452,-2.191658
max,17.749851,3.570765,17.066487,16.178942,7.912809,21.445837,-0.424033,13.846083,4.789989,16.326455,9.166066,5.259430


In [27]:
# Qualitatives
df.language.value_counts()

US    165
UK     45
GE     30
IT     30
FR     30
ES     29
Name: language, dtype: int64

#### 3. Visualiser les targets, et que remarquez vous ?
(combien d'instances de chaque valeur de la target)

In [28]:
# Nombre de valeurs
df.language.nunique()

6

In [29]:
# Valeurs
df.language.unique()

array(['ES', 'FR', 'GE', 'IT', 'UK', 'US'], dtype=object)

In [30]:
# Fréquences de chaque valeurs
pd.crosstab(df.language, "freq") #.plot(kind="hist")

col_0,freq
language,
ES,29
FR,30
GE,30
IT,30
UK,45
US,165



#### 4. Notre variable target (Y) est 'language', Récuprer X et y à partir du jeu de données 

In [31]:
X = df.iloc[:,1:]  # type (X) -> DataFrame
# X = df.language # type (X) -> Series
# X = df["language"] # type (X) -> Series
X

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12
0,7.071476,-6.512900,7.650800,11.150783,-7.657312,12.484021,-11.709772,3.426596,1.462715,-2.812753,0.866538,-5.244274
1,10.982967,-5.157445,3.952060,11.529381,-7.638047,12.136098,-12.036247,3.491943,0.595441,-4.508811,2.332147,-6.221857
2,7.827108,-5.477472,7.816257,9.187592,-7.172511,11.715299,-13.847214,4.574075,-1.687559,-7.204041,-0.011847,-6.463144
3,6.744083,-5.688920,6.546789,9.000183,-6.924963,11.710766,-12.374388,6.169879,-0.544747,-6.019237,1.358559,-6.356441
4,5.836843,-5.326557,7.472265,8.847440,-6.773244,12.677218,-12.315061,4.416344,0.193500,-3.644812,2.151239,-6.816310
...,...,...,...,...,...,...,...,...,...,...,...,...
324,-0.525273,-3.868338,3.548304,1.496249,3.490753,5.849887,-7.747027,9.738836,-11.754543,7.129909,0.209947,-1.946914
325,-2.094001,-1.073113,1.217397,-0.550790,2.666547,7.449942,-6.418064,10.907098,-11.134323,6.728373,2.461446,-0.026113
326,2.116909,-4.441482,5.350392,3.675396,2.715876,3.682670,-4.500850,11.798565,-12.031005,7.566142,-0.606010,-2.245129
327,0.299616,0.324844,3.299919,2.044040,3.634828,6.693840,-5.676224,12.000518,-11.912901,4.664406,1.197789,-2.230275


In [32]:
y = df.iloc[:,:1] # type(y) -> DataFrame
# y = df.iloc[:,0] # type(y) -> Series
# y = df.select_dtypes('number') # type(y) -> DataFrame
y


,language
0,ES
1,ES
2,ES
3,ES
4,ES
...,...
324,US
325,US
326,US
327,US


#### 5. Diviser la DataSet en donneés d'apprentissage et de test (20% pour le test)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 20/100, random_state = 0)

In [34]:
for split in [X_train, X_test, y_train, y_test]:
  print(split.shape)

(263, 12)
(66, 12)
(263, 1)
(66, 1)


#### 6. Appliquer une normalisation centrée-réduite aux données en utilisant "StandardScaler"

Former votre modèle de normalisation


In [46]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train.copy())

StandardScaler()

Utiliser la classe ".mean_" et ".scale_" pour afficher la moyenne et l'écart-type, respectivement.

In [47]:
scaler.mean_

array([ 5.45011894, -4.28675644,  2.6865847 ,  7.09095867, -5.53779014,
        9.83197748, -9.39524525,  5.20175913, -1.39374422, -2.1926616 ,
        2.47055528, -3.92454806])

In [48]:
scaler.scale_

array([5.04951755, 3.33613672, 3.50535463, 4.42568847, 4.64351478,
       3.65513597, 2.44825969, 2.68760927, 3.76929674, 5.08995082,
       3.27858839, 3.09421694])

Appliquer la normalisation sur la base d'apprentissage et la base de test sans écraser les données brutes.

In [49]:
X_train_scaled = scaler.transform(X_test.copy())


Vérifier que la moyenne se rapproche à 0

In [50]:
X_train_scaled.mean()

-0.013672107490727323

Vérifier que l'écart-type se rapproche à 1

In [51]:
X_train_scaled.std()

1.0122705066784288

#### 5. Apprentissage supervisé SVM

Réaliser le Pipeline expérimental

- Data
- Analyse, visualisation et prétraitement (si nécessaire)
- Préparation des données
- Apprentissage et Normalisation
- Configuration des SVMs avec différent paramétrage
- Apprentissage et Test
- Matrice de confusion
- Mesure de performance
- Conclusion

## Etude de cas
Un client cherche un produit pour identifier que l’accent italien et l’accent français.
Quel est votre proposition pour ce client ?

J'ai programmé une gonction pour vous faciliter la classification d'un nouveau Voice, la fonction est "rec_accent".
Les inputs de la fonction sont :
- classifier : le modèle SVM ou autre.
- Record=True : pour faire votre propre enregistrement.
- Record=False : pour charger un enregistrement existant.

Output est votre reconnaissance.

In [ ]:
import sounddevice as sd #sinon pip install sounddevice
import time #sinon pip install python-time
import scipy.io.wavfile as wav
from python_speech_features import mfcc #sinon pip install python_speech_features==0.4
import tkinter as tk #sinon pip install tk
from tkinter import filedialog

def rec_accent(classifier, Record=True):
    if Record == True :
        #Enregistrement
        print("Attention, l'enregistrement commence dans :")
        (rate,sig) = wav.read("beep-04.wav")
        sd.play(sig, rate)
        for i in range(0,6):
            time.sleep(1)
            print(5-i)
        time.sleep(1)
        freq = 44100
        duration = 1
        recording = sd.rec(int(duration * freq), samplerate=freq, channels=2)
        print("Parler maintenant ...!!")
        sd.wait()  
        print("Fin...")
    else :
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilename()
        (rate,recording) = wav.read(file_path)
            
    # Caractérisation
    mfcc_feat = mfcc(recording,rate, numcep=12)
    mfcc_feat = np.expand_dims(np.mean(mfcc_feat, axis=0), axis=0)
    pred = classifier.predict(mfcc_feat)
    print('------------------')
    print('Accent : ', pred[0])
    print('------------------')

In [21]:
rec_accent(trained_classifier, Record=False)

------------------
Accent :  US
------------------


C:\Users\isen\anaconda3\envs\meab\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearSVC was fitted with feature names
  warnings.warn(
